# 图像处理与计算机视觉研讨课课程大作业
## 陈真宇 2022K8009991010
在课程中我们已经学习过图像语义分割，通常的工具有DeepLabV3+等。我们还学习过三维重建，通常的工具有colmap，hloc等。在这个大作业中，我们将结合这两个技术，实现语义三维重建。本实验将这两个技术结合起来，做出一种基于DeepLabV3+和colmap的稀疏点云语义三维重建的尝试。
## 任务要求
1）基于图像进行语义分割或目标检测，mask影响建模物体，比如行人、汽车等；

2）基于mask的图像进行特征检测匹配和场景三维建模，相同语义点云用相同颜色
表示，语义标签至少包括：建筑、地面和植被，无法识别语义的点云用黑色表示；

3）测试数据集包括1组给定数据集和1组自采数据集
## 任务分析
首先我们结合课上老师给的提示，我们决定采用DeepLabV3+网络进行语义分割，然后使用colmap进行三维重建。我们的任务分为两个部分：

1）语义分割：使用DeepLabV3+网络进行语义分割，将图像中的物体分割出来；

2）三维重建：使用colmap处理图像，得到稀疏点云并且会返回一个output的txt文件夹，然后通过修改points3D.txt文件，将语义信息对应的颜色信息写入，然后再用colmap进行三维重建，得到带有语义信息的三维重建结果。

如此，我们可以发现这个实验的关键在于如何将deeplab的语义信息和colmap三维重建的文件做对齐，如何为3D点写入rgb颜色，以及当不同的原始图片的分割不一致的时候我们如何处理信息从而保证较高的语义重建精度。
## 执行环境
本demo使用的操作系统是linux系统，具体为ubuntu22.04，环境依赖详见requirements.txt文件。
## 算法总结
- 语义分割：我们使用DeepLabV3+网络进行语义分割，将图像中的物体分割出来。我们使用的是deeplabv3+的预训练模型。
- 三维重建：我们使用colmap进行三维重建，特征点提取主要测试了SIFT方法，
## 项目结构
```
project_root/
├── data/
│   └── given/                  # 给定数据集图片
│   └── self_collected/         # 自采数据集图片
├── output/
│   └── colmap/                 # colmap输出文件夹
│       └── given/
│           ├── cameras.txt
│           ├── images.txt
│           └── points3D.txt
│       └── self_collected/
│           ...
├── semantic_data/              # 使用deeplabv3获得的语义信息
│   └── given/
│    ...
├── semantic_output/            # 将语义信息整理进colmap输出文件夹
│   └── given/
│    ...
├── utils/                      # 工具函数
│   └── update_3d_color.py      # 更新3d点云的颜色
│   └── video2pic.py            # 从视频中抽帧
├── sh/                         # 存放linux上运行的shell脚本
│    ...
├── pic/                        # 存放docs中用到的图片
│    ...
├── docs/                       # 存放文档
│    ...
├── README.md                   # 项目说明
├── requirements.txt            # 项目依赖
├── demo.ipynb                  # 演示代码和说明
```

## 实验过程
### 使用DeepLabV3+算法进行语义分割
我们首先从github上clone deeplabv3+的代码，然后使用给定数据集和自采数据集进行语义分割。我们使用的是deeplabv3+的预训练模型，因此我们只需要将图片输入到模型中，然后得到语义分割的结果。我们将语义分割的结果保存在semantic_data文件夹中。

为了更加方便的实现这两个操作和复现实验结果，我们将这两个操作封装成了两个shell脚本，分别是`sh/given_predict.sh`和`sh/self_predict.sh`。我们可以通过在demo中运行这两个脚本来实现语义分割。对于windows用户，可以复制`sh/given_predict.sh`和`sh/self_predict.sh`中的命令，手动在命令行中运行。


In [1]:
!git clone https://github.com/VainF/DeepLabV3Plus-Pytorch

Cloning into 'DeepLabV3Plus-Pytorch'...
remote: Enumerating objects: 705, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 705 (delta 25), reused 14 (delta 14), pack-reused 668 (from 2)
Receiving objects: 100% (705/705), 8.26 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (380/380), done.


#### 给定数据集上：
因为在第二章图像分割作业中，我以往已经跑通过这个repo，所以我理所当然地以为这次也应该很顺利，但是后来发现对于给定的数据集，我在这个repo上既没有报错，也没有返回任何有效的语义分割结果，我一时间陷入了困境，我多次尝试过重头装环境，降低pytorch版本，重新git clone repo，但都无济于事，后来在阅读代码时发现，predict.py中的这一行用于读取文件的代码：
```python
        ...
        for ext in ['png', 'jpeg', 'jpg', 'JPEG']:
        ...
```
并不支持给定数据中的`JPG`格式，'JPG'本身也不是一个非常常见的图片后缀名。这也解释了为什么既没有报错也没有返回值，因为程序读取到这是一个空目录，直接结束了进程。我将这一行代码改为：
```python
        ...
        for ext in ['png', 'jpeg', 'jpg', 'JPEG', 'JPG']:
        ...
```
之后就可以正常运行了。我随后注意到github上`#144`issue有人提到了no results的问题，我提出了我的解决方法，希望可以帮助到更多看到这条issue的人。

需要说明的是，这的我们的数据集都是室外的街景，所以预训练模型的选择上自然选择了基于`cityscapes`数据集的预训练模型。

In [1]:
# 将data文件夹复制到DeepLabV3Plus-Pytorch文件夹下
!cp -r './data' './DeepLabV3Plus-Pytorch'
!cp './sh/given_predict.sh' './DeepLabV3Plus-Pytorch'
!cd DeepLabV3Plus-Pytorch && bash given_predict.sh
!cd ..
!cp -r './DeepLabV3Plus-Pytorch/semantic_data/given' './semantic_data'


Device: cuda
Resume model from checkpoints/best_deeplabv3plus_mobilenet_cityscapes_os16.pth
100%|█████████████████████████████████████████| 128/128 [01:08<00:00,  1.87it/s]


这一步的结果如下被放到了`semantic_data/given`文件夹中。这里我们给出一个对比图：
<p float="left">
  <img src="data/given/P1180173.JPG" width="400" />
  <img src="semantic_data/given/P1180173.png" width="400" />
</p>
这是一张比较有代表性的图片，可以看到，虽然图中的人呈现坐姿，夹在图片两种语义（building, truck）之间并且只在图像空间中占据很小的面积，但是deeplabv3+仍然成功地将其分割出来，并且赋予了正确的语义标签。

#### 自采数据集：
首先我在整个流程上测试了自采数据集，由于拍摄的是小区内的建筑，而小区内的楼层过高导致特征点匹配难以进行，在街道上拍摄的建筑物显得很局促，我尝试了多次，但是都没有得到很好的结果。

在老师的许可下，我使用了公开数据集`Buddah tooth relic temple, Singapore`进行测试。这个数据集包含了一些建筑物，地面和植被，以及很多来往的行人，我认为这个数据集可以很好的展示我们的任务要求。

后面同样的，我们运行脚本，得到了语义分割的结果。这里展示一张对比图

<p float="left">
  <img src="data/self_collected/DSC_0436.JPG" width="400" />
  <img src="semantic_data/self_collected/DSC_0436.png" width="400" /> 
</p>
这是一张典型的具有很高复杂度的图像，因为其中的语义种类十分复杂。可以看到，person，car，wall，sky，ground等物体都被成功分割出来了，并且颜色也比较准确，这为我们后续的三维重建提供了很好的语义信息。其中sidewalk和ground的区分也比较明显，这很难能可贵，但是也有一些问题，比如这里的路灯被分类到了traffic light中，但是也无可厚非了，因为cityscapes数据集中的类别种类有限，而且路灯和交通灯在形状上也有很多相似之处。


In [2]:
!cp -r './data/self_collected' './DeepLabV3Plus-Pytorch/data'
!cp './sh/self_predict.sh' './DeepLabV3Plus-Pytorch'
!cd DeepLabV3Plus-Pytorch && bash self_predict.sh
!cd ..
!cp -r './DeepLabV3Plus-Pytorch/semantic_data/self_collected' './semantic_data'

Device: cuda
Resume model from checkpoints/best_deeplabv3plus_mobilenet_cityscapes_os16.pth
  0%|                                                   | 0/162 [00:00<?, ?it/s]/home/chenzhenyu/anaconda3/envs/cu121py312/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|█████████████████████████████████████████| 162/162 [00:33<00:00,  4.83it/s]


### 使用colmap进行三维建模
#### 给定数据集上：
我使用colmap进行三维重建，在第一次做的时候我用了课上用过的colmap windows no cuda版本，可能是由于没有cuda加速，虽然gpu占用率几乎都在100%，但是在matching的时候还是尤其耗时，在我的3060上需要花费30分钟左右。我们使用课上教学的流程创建项目，下面的是过程截图：

![matching](pic/given/matching.png)
<p float="left">
  <img src="pic/given/reconstruction1.png" width="400" />
  <img src="pic/given/reconstruction2.png" width="400" /> 
</p>

#### 自采数据集上：
由于自采数据集上，图片更多了，所以耗费的时间也相对更多。这里展示一下reconstruction的过程：

![matching](pic/self_collected/matching.png)
<p float="left">
  <img src="pic/self_collected/reconstruction1.png" width="400" />
  <img src="pic/self_collected/reconstruction2.png" width="400" />
</p>

### 语义信息与三维点云对齐
这里主要介绍我们的算法核心部分，也就是如何将语义信息和colmap输出的三维点坐标信息对齐。为此我们编写了`utils/update_3d_color.py`。下面是代码讲解：
- 首先我们查看`DeepLabV3Plus-Pytorch/datasets/cityscapes.py`文件，详细阅读代码是如何将语义和掩码颜色的对应关系，然后我们将这个对应关系通过字典的形式应用到我们的函数中：
  ![color](pic/code1.png)
- 然后我们读取colmap输出的`points3D.txt`文件，这个文件中包含了所有的三维点信息，我们可以从中拿到三维点id和原始图像以及原始图像关系，然后我们遍历读出`images.txt`文件，找到对应的图片，读取这个三维点在这个图片中的二维坐标，然而由于sift的特性，这个坐标通常是一个浮点数，我们需要将其使用`round`函数取整，然后我们就可以在先前输出的`semantic_data`文件夹中找到对应的图片再找到对应的二维坐标，这个二维坐标的颜色信息便可以反应这张图象上给出的语义信息，由于有一个三维点可能来自于多个图片，而这些图片的语义分割结果不见得相同，所以我们遍历这些图片，维护一个字典记录单点的语义和出现次数，然后用counter找到出现次数最多的语义，我们认为这个语义就是这个三维点的语义信息。
  ```python
  for (i, point_line) in zip(range(len(points_lines)), points_lines):
        # points3d_id, x, y, z, r, g, b, error, (image_id, points2d_id)
        point_line_list = point_line.strip().split()
        image_id2points2d_id = {point_line_list[i]: point_line_list[i + 1] for i in range(8, len(point_line_list), 2)}
        cls_list = []
        for image_id, points2d_id in image_id2points2d_id.items():
            image_id = int(image_id)
            image_line = [image_lines[2*image_id-2], image_lines[2*image_id-1]]
            image_line_list = [image_line[0].strip().split(), image_line[1].strip().split()]
            image_name = image_line_list[0][-1]
            image_name = image_name.split('.')[0] + '.png'
            image = name2image[image_name]
            points_2d_coordinate = (round(float(image_line_list[1][int(points2d_id)*3])), round(float(image_line_list[1][int(points2d_id)*3+1])))
            bgr = image[points_2d_coordinate[1], points_2d_coordinate[0]]
            rgb = tuple(reversed(bgr))
            cls_list.append(rgb2cls['citiscapes'][rgb])
        cls_counter = Counter(cls_list)
        # choose the most common class as the final class
        cls = cls_counter.most_common(1)[0][0]
        rgb = cls2rgb['citiscapes'][cls]
        point3d_id2rgb[point_line_list[0]] = rgb
  ```
  这里大循环是遍历三维点，小循环是遍历这个三维点在不同图片上的二维坐标，
- 然后我们将所有的三维点的颜色信息写入`points3D.txt`文件中，这样我们就可以在colmap中看到带有语义信息的三维重建目录了。写入代码较简单，详见`utils/update_3d_color.py`。
- 最后我们在demo中调用这个函数，便可以得到带有语义信息的三维重建结果。

In [1]:
from utils.update_3d_color import update_color
update_color('output/colmap/given/cameras.txt',
             'output/colmap/given/images.txt',
             'output/colmap/given/points3D.txt',
             'semantic_data/given',
             'semantic_output/given')

In [3]:
from utils.update_3d_color import update_color
update_color('output/colmap/self_collected/cameras.txt',
             'output/colmap/self_collected/images.txt',
             'output/colmap/self_collected/points3D.txt',
             'semantic_data/self_collected',
             'semantic_output/self_collected')

#### 时间复杂度分析
上述过程较为耗时，我们不妨分析一下整个过程的时间复杂度，假设我们有n张图片，则最后得到的三维点大致为O(k*n)个，而每个三维点又在O(n)张图片上出现，所以整个过程的时间复杂度为O(n^2)。优化过程中，尽管我们一次性读取出了所有的图片和文件，这样做可以在常数意义下减小复杂度，但是由于我们需要遍历所有的三维点，所以还是相当耗时的，我的12代i9处理器在每一个数据集上大概要花费10来分钟。
### 使用colmap呈现语义分割结果：
我们可以使用colmap的`colmap gui`来查看三维重建结果，直接导入model，给定数据集上的结果：
<p float="left">
  <img src="pic/given/semantic1.png" width="400" />
  <img src="pic/given/semantic2.png" width="400" />
</p>
可以看到，我们的三维重建结果中，不同的物体被赋予了不同的颜色，这些颜色是由语义分割结果决定的，在第二张图中，正对应着我们刚刚展示的语义分割结果，可以看到，尽管门前看书的人在图像空间中占据的面积很小，但相关的3d点云仍然赋予了粉色，即人的语义标签。

在自采数据集上的结果：
<p float="left">
  <img src="pic/self_collected/semantic1.png" width="400" />
  <img src="pic/self_collected/semantic2.png" width="400" />
</p>
可以从图中看出，尽管稀疏点云很杂乱，但是仍然被赋予了较好的语义标签，这也说明了我们的算法的有效性。由于sift的特性，提取的特征点不够多，特别是人物和车辆上，所以这里的点云很难直观反映出这些物体的形状，但是我们的算法仍然成功地将这些物体分割出来了。

## 结果分析和讨论
### 算法的优点
巧妙的结合了DeepLabV3+和colmap，实现了语义三维重建的任务，这是一个新颖的尝试，也是一个有趣的实验。我们的算法在给定数据集上取得了较好的效果，成功地将语义信息和三维重建结果结合起来，这对于一些特定的应用场景是非常有用的。我们的算法还通过投票法，可以很好地处理不同图片的语义分割不一致的问题，有一定的鲁棒性。

### 可能的缺点
由于SIFT算法的固有问题，这导致了我们的稀疏重建结果很难反映出地面的点云，这是一个比较大的缺点，这相当于我们很大程度上在点云结果上缺失了地面的语义信息。

另外，我们的投票算法虽然可以很好地处理不同图片的语义分割不一致的问题，但是也有一定的局限性，比如在一些特殊情况下，可能会出现错误标签大于正确标签的情况，这是一个需要改进的地方。

### 改进想法
- 我们可以尝试使用更好的特征点提取算法，比如应用hloc工具，其中提供了disk特征提取算法，通过我们以往的实践得知，disk点不光会在意建筑物主体特征，还有兼顾地面的优点，而对于匹配算法我们可以尝试其中的lightglue算法，这个算法在匹配的时候会对光照有更大的鲁棒性，这样我们就可以更好地克服拍摄环境树荫遮挡等问题的影响。
- 虽然hloc是在我能力范围之内的，但是由于时间和计算资源的问题，过多的特征点和特征点匹配会带来更大的计算量，在整合语义信息的时候也会变得更加困难。我们可以尝试改进一下投票算法，由于DeepLabV3+的输出是一个概率分布，我们可以尝试获取这个概率作为置信度，从而取前3名的语义标签，然后再基于置信度投票，这样既增加了算法的鲁棒性，也减少了计算量（具体而言变为了一个线性时间复杂度的算法）
- 为了获取更好的重建结果，可能我们可以结合语义信息和稠密重建，这样我们可以更好地获取地面的点云

## 总结
通过这次实验，我们成功地结合了DeepLabV3+和colmap，实现了语义三维重建的任务。我们的算法在给定数据集上取得了较好的效果，成功地将语义信息和三维重建结果结合起来，这对于一些特定的应用场景是非常有用的。算法还通过投票法，可以很好地处理不同图片的语义分割不一致的问题，有一定的鲁棒性。但是算法还有一些缺点，比如SIFT算法的固有问题导致了我们的稀疏重建结果很难反映出地面的点云。为此，本文提出了一些改进的想法，比如尝试使用更好的特征点提取算法，改进投票算法等。总的来说，这次实验是一次很有意义的尝试，在不额外训练网络的情况下，我实现了这个任务。这也是一次很有趣的实验，我在这次实验中学到了很多，也发现了很多问题，如果有机会，我会继续精进这个算法，希望能够取得更好的效果。

## 参考文献
- Sch\"{o}nberger, Johannes Lutz and Frahm, Jan-Michael. "Structure-from-Motion Revisited." *Conference on Computer Vision and Pattern Recognition (CVPR)*, 2016.
- Sch\"{o}nberger, Johannes Lutz, Zheng, Enliang, Pollefeys, Marc, and Frahm, Jan-Michael. "Pixelwise View Selection for Unstructured Multi-View Stereo." *European Conference on Computer Vision (ECCV)*, 2016.
- Chen, Liang-Chieh, Zhu, Yukun, Papandreou, George, Schroff, Florian, and Adam, Hartwig. "Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation." *arXiv preprint arXiv:1802.02611*, 2018.

